<a href="https://colab.research.google.com/github/sameer203/ANN-Implementation/blob/main/Basic_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
ROOT = "/content/drive/MyDrive/GAN"
os.chdir(ROOT)

In [ ]:
os.getcwd()

'/content/drive/My Drive/GAN'

### Import Libraries

In [ ]:
import torch, pdb
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

### Visualization Function

In [ ]:
def show(tensor, ch=1, size=(28,28), num=16):
  data = tensor.detach().cpu().view(-1, ch, *size)
  grid = make_grid(data[:num], nrows=4).permute(1,2,0)
  plt.imshow(grid)
  plt.show()

### Setup of the main parameters and hyperparamaters

In [ ]:
epochs = 500
cur_step = 0
info_step = 300
mean_gen_loss = 0
mean_disc_loss = 0

z_dim = 64
lr = 0.00001
loss_func = nn.BCEWithLogitsLoss()

bs = 128
device = "cuda"

dataloader = DataLoader(MNIST(".", download=True, transform=transforms.ToTensor()), shuffle=True, batch_size=bs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Model - Generator

In [ ]:
def genBlock(inp,out):
  return nn.Sequential(
      nn.Linear(inp, out),
      nn.BatchNorm1d(out),
      nn.ReLU(inplace=True)
  )

class Generator(nn.Module):
  def __init__(self, z_dim=64, i_dim=784, h_dim=128):
    super().__init__()
    self.gen = nn.Sequential(
        genBlock(z_dim, h_dim),
        genBlock(h_dim, h_dim*2),
        genBlock(h_dim*2, h_dim*4),
        genBlock(h_dim*4, h_dim*8),
        nn.Linear(h_dim*8, i_dim),
        nn.Sigmoid()
    )

  def forward(self, noise):
    return self.gen(noise)


def gen_noise(number, z_dim):
  return torch.randn(number, z_dim).to(device)

### Model - Discriminator

In [ ]:
def discBlock(inp,out):
  return nn.Sequential(
      nn.Linear(inp, out),
      nn.LeakyReLU(0.2)
  )

class Discriminator(nn.Module):
  def __init__(self, i_dim=784,h_dim=256):
    super().__init__()
    self.disc = nn.Sequential(
        disBlock(i_dim, h_dim*4),
        disBlock(h_dim*4, h_dim*2),
        disBlock(h_dim*2, h_dim),
        nn.Linear(h_dim, 1)
    )

  def forward(self, noise):
    return self.disc(image)